In [1]:
%%time
# Assembling RoM and PoP for option chains
# Should be run only after all prior programs (01, 02, 03 and 04)

# STATUS: Completed
# Runtime: 45 sec

import pandas as pd
import datetime
from scipy.stats import norm
import numpy as np
import math

# For Black Shcoles 
from math import sqrt, exp, log, erf

#******         Paths and variables         ****
#_______________________________________________

datapath = r'./zdata/'
today = datetime.datetime.now().date()

#*****            Set up the Limits         ****
#_______________________________________________

sd_days = 252 # no of days for the standard deviation

# Read the pickles
df_ohlc = pd.read_pickle(datapath+'df_ohlc.pkl')
df_options = pd.read_pickle(datapath+'df_nse_options.pkl')
df_nse_ib = pd.read_pickle(datapath+ 'df_underlying.pkl')

#*****              Black-Scholes         ******
#_______________________________________________

# Ref: - https://ideone.com/fork/XnikMm - Brian Hyde

def get_bsm(undprice, strike, time, rate, sigma, divrate):
    ''' Gets Black Scholes output
    Args:
        (undprice) : Current Stock Price in float
        (strike)   : Strike Price in float
        (time)     : Days to expiration in float
        (rate)     : Time until expiry in days
        (sigma)    : Standard Deviation of stock's return in float
        (divrate)  : Dividend Rate in float
    Returns:
        (delta, call_price, put_price) as a tuple
    '''
    #statistics
    sigTsquared = sqrt(time/365)*sigma
    edivT = exp((-divrate*time)/365)
    ert = exp((-rate*time)/365)
    d1 = (log(undprice*edivT/strike)+(rate+.5*(sigma**2))*time/365)/sigTsquared
    d2 = d1-sigTsquared
    Nd1 = (1+erf(d1/sqrt(2)))/2
    Nd2 = (1+erf(d2/sqrt(2)))/2
    iNd1 = (1+erf(-d1/sqrt(2)))/2
    iNd2 = (1+erf(-d2/sqrt(2)))/2

    #Outputs
    callPrice = round(undprice*edivT*Nd1-strike*ert*Nd2, 2)
    putPrice = round(strike*ert*iNd2-undprice*edivT*iNd1, 2)
    delta = Nd1
    
    return (callPrice, putPrice, delta)

#******      Get the Volatility from Annual Standard Deviations  ****
#____________________________________________________________________

# get the max and min of ohlc for each date
df_ohlc['maxp'] = df_ohlc.loc[:, ['O', 'H', 'L', 'C']].max(1) # Max price for calls
df_ohlc['minp'] = df_ohlc.loc[:, ['O', 'H', 'L', 'C']].min(1) # Min price for puts

# Compute the annual standard deviation for calls and puts
df_cSD = df_ohlc[(df_ohlc.D > df_ohlc.D.max()- \
         datetime.timedelta(days=sd_days))].groupby('ibSymbol') \
        ['ibSymbol', 'maxp'].std(ddof=0).rename(columns={'maxp': 'cASD'})

df_pSD = df_ohlc[(df_ohlc.D > df_ohlc.D.max()- \
         datetime.timedelta(days=sd_days))].groupby('ibSymbol') \
        ['ibSymbol', 'minp'].std(ddof=0).rename(columns={'minp': 'pASD'})

df_aSD = df_ohlc[(df_ohlc.D > df_ohlc.D.max()- \
         datetime.timedelta(days=sd_days))].groupby('ibSymbol') \
        ['ibSymbol', 'C'].std(ddof=0).rename(columns={'C': 'ASD'})

df_mean = df_ohlc[(df_ohlc.D > df_ohlc.D.max()- \
         datetime.timedelta(days=sd_days))].groupby('ibSymbol') \
        ['ibSymbol', 'C'].mean().rename(columns={'C': 'Mean'})

df_SD = pd.concat(objs=[df_aSD, df_cSD, df_pSD, df_mean], axis=1).reset_index()

# Convert Expiry column into datetime
df_options.Expiry = pd.to_datetime(df_options.Expiry)

# get the days to expiry for the options
df_options['DTE'] = (df_options.Expiry - datetime.datetime.now()).dt.days

# Determine put or call - option type
df_options['Type'] = np.where(df_options.Strike > df_options.undPrice, 'C', 'P')

# Compute risk-free rate (r) from 91 day T-bills
rate_url = 'https://rbi.org.in/home.aspx'

li = pd.read_html(rate_url)
li_df = li[4].rename(columns = {0: 'Cat', 1: 'Values'})
li_val = li_df.loc[li_df.Cat == '91 day T-bills', 'Values']
r = float((str(li_val).split('\n')[0].split('%')[0].split(' ')[-1:])[0])/100

# Add ibSymbol, margin, lots and standard deviations to df_options
df = df_options.merge(df_nse_ib, on='nseSymbol').merge(df_SD, on='ibSymbol')
# Remove options where DTE < 0 and correct the 0 DTE to 1 (for last-day-options)
df = df[df.DTE >= 0].reset_index(drop=True)

df.loc[df.DTE == 0, 'DTE'] = 1    # Corrects date to 1 day for last day of expiry

# Add risk-free rate 
df['Rate'] = r

df['Sigma'] = df.ASD/df.Mean     # Sigma is volatility measure
df['Dividend'] = 0   # Dividend to be put in future

# ****   Compute Black-Scholes and put it to a dataframe   ***
df_bsm = pd.DataFrame(list(np.vectorize(get_bsm)(df.undPrice, df.Strike, df.DTE, df.Rate, df.Sigma, df.Dividend))).T.rename(columns = {0: 'cPrice', 1: 'pPrice', 2: 'Delta'})

# The mega dataframe
df = df.join(df_bsm, how='outer')

# Pickle the output
df.to_pickle(datapath+'df_main.pkl')

Wall time: 31.5 s


In [3]:
df

,nseSymbol,Expiry,Strike,undPrice,cOI,cChnginOI,cVolume,cIV,cLTP,cNetChng,...,ASD,cASD,pASD,Mean,Rate,Sigma,Dividend,cPrice,pPrice,Delta
0,ACC,2018-12-27,1140.0,1508.35,NaN,NaN,NaN,NaN,NaN,NaN,...,28.019051,27.317069,24.506391,1470.916667,0.067706,0.019049,0,372.78,0.00,1.000000e+00
1,ACC,2018-12-27,1160.0,1508.35,NaN,NaN,NaN,NaN,NaN,NaN,...,28.019051,27.317069,24.506391,1470.916667,0.067706,0.019049,0,352.86,0.00,1.000000e+00
2,ACC,2018-12-27,1180.0,1508.35,NaN,NaN,NaN,NaN,NaN,NaN,...,28.019051,27.317069,24.506391,1470.916667,0.067706,0.019049,0,332.94,0.00,1.000000e+00
3,ACC,2018-12-27,1200.0,1508.35,NaN,NaN,NaN,NaN,NaN,NaN,...,28.019051,27.317069,24.506391,1470.916667,0.067706,0.019049,0,313.02,0.00,1.000000e+00
4,ACC,2018-12-27,1220.0,1508.35,NaN,NaN,NaN,NaN,NaN,NaN,...,28.019051,27.317069,24.506391,1470.916667,0.067706,0.019049,0,293.09,0.00,1.000000e+00
5,ACC,2018-12-27,1240.0,1508.35,NaN,NaN,NaN,NaN,NaN,NaN,...,28.019051,27.317069,24.506391,1470.916667,0.067706,0.019049,0,273.17,0.00,1.000000e+00
6,ACC,2018-12-27,1260.0,1508.35,NaN,NaN,NaN,NaN,NaN,NaN,...,28.019051,27.317069,24.506391,1470.916667,0.067706,0.019049,0,253.25,0.00,1.000000e+00
7,ACC,2018-12-27,1280.0,1508.35,NaN,NaN,NaN,NaN,NaN,NaN,...,28.019051,27.317069,24.506391,1470.916667,0.067706,0.019049,0,233.33,0.00,1.000000e+00
8,ACC,2018-12-27,1300.0,1508.35,NaN,NaN,NaN,NaN,NaN,NaN,...,28.019051,27.317069,24.506391,1470.916667,0.067706,0.019049,0,213.40,0.00,1.000000e+00
9,ACC,2018-12-27,1320.0,1508.35,NaN,NaN,NaN,NaN,NaN,NaN,...,28.019051,27.317069,24.506391,1470.916667,0.067706,0.019049,0,193.48,0.00,1.000000e+00
